In [61]:
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.augmentations.transforms import *
from albumentations.pytorch import ToTensorV2
from mediapipe import solutions

from PIL import Image

folder_path = r"D:\original_data_set\ARSL-No-BackGround-V2"
target_count = 500
image_size = (224, 224)

mp_hands = solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1)

# Define augmentation pipeline
augment = A.Compose([
    A.Rotate(limit=20, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RGBShift(p=0.2),
    A.MotionBlur(p=0.1),
])

for class_name in os.listdir(folder_path):
    class_dir = os.path.join(folder_path, class_name)
    if not os.path.isdir(class_dir):
        continue

    images = [f for f in os.listdir(class_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    current_count = len(images)

    if current_count >= target_count:
        continue
    else:
        needed = target_count - current_count
        print(f"{class_name}: augmenting {needed} images...")

    i = 0
    for img_name in images:
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if not results.multi_hand_landmarks:
            continue  # Skip if no hand detected

        for _ in range(5):  
            augmented = augment(image=img_rgb)
            aug_img = augmented['image']
            aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
            save_path = os.path.join(class_dir, f"aug_{i}_{img_name}")
            cv2.imwrite(save_path, aug_img_bgr)
            i += 1

            if i >= needed:
                break
        if i >= needed:
            break

Ain: augmenting 28 images...


In [34]:
import random

folder_path_2 = r"D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2"

target = 250

for class_name in os.listdir(folder_path_2):
    class_dir = os.path.join(folder_path_2, class_name)
    if not os.path.isdir(class_dir):
        continue
        
    images = []
    for img_name in os.listdir(class_dir):
        if img_name.endswith(('.jpg', '.jpeg', '.png')):
            images.append(os.path.join(class_dir, img_name))
    if len(images) > target:
        images_to_remove = random.sample(images, len(images) - target)
        for img_path in images_to_remove:
            os.remove(img_path)
            print(f"Removed {img_path}")

Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_225.jpeg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_195.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_178.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_28.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_148.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_198.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Al\Al_271.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_65.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_133.jpeg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_174.jpeg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_51.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_81.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_231.jpg
Removed D:\arabic_dataset_for_pre\ARSL-No-BackGround-V2\Alef\Alef_1

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (224, 224)  
data_dir = r"D:\original_data_set\ARSL-No-BackGround-V2"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Set aside 20% for validation
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle =True
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle =False
)

KeyboardInterrupt: 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3), padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    # Dropout(0.2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    # Dropout(0.2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    # Dropout(0.2),

    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax') 
])

c:\Users\Samer\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'train_data' is not defined

In [66]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

early_stop = EarlyStopping(
    monitor='val_accuracy',   
    patience=7,               
    restore_best_weights=True      
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=4,
    min_lr=1e-6
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, validation_data=val_data, epochs=30, callbacks=[early_stop, reduce_lr])

Epoch 1/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1017s 3s/step - accuracy: 0.2851 - loss: 2.2622 - val_accuracy: 0.3636 - val_loss: 1.9367 - learning_rate: 0.0010
Epoch 2/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1016s 3s/step - accuracy: 0.3968 - loss: 1.8722 - val_accuracy: 0.4043 - val_loss: 1.7991 - learning_rate: 0.0010
Epoch 3/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1018s 3s/step - accuracy: 0.5049 - loss: 1.5066 - val_accuracy: 0.5100 - val_loss: 1.5151 - learning_rate: 0.0010
Epoch 4/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1012s 3s/step - accuracy: 0.6053 - loss: 1.2216 - val_accuracy: 0.5806 - val_loss: 1.2591 - learning_rate: 0.0010
Epoch 5/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1024s 3s/step - accuracy: 0.6661 - loss: 1.0184 - val_accuracy: 0.6193 - val_loss: 1.1473 - learning_rate: 0.0010
Epoch 6/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1020s 3s/step - accuracy: 0.7239 - loss: 0.8524 - val_accuracy: 0.5113 - val_loss: 1.4959 - learning_rate: 0.0010
Epoch 7/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1018s 3s/step - accuracy: 0.7606 - l

In [1]:
model.save(r"D:\original_data_set\ARSL-No-BackGround-V2\model.h5")

NameError: name 'model' is not defined